# 1) 순환 신경망
## 2. 케라스로 RNN 구현하기

In [7]:
from tensorflow.keras.layers import SimpleRNN

# 책에선 없지만 없으면 model 안됨
from tensorflow.keras.models import Sequential
model = Sequential()

model.add(SimpleRNN(hidden_units))

NameError: ignored

In [8]:
# 추가 인자를 사용할 때
model.add(SimpleRNN(hidden_units, input_shape=(timesteps, input_dim)))

# 다른 표기
model.add(SimpleRNN(hidden_units, input_length=M, input_dim=N))

NameError: ignored

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [10]:
# batch_size 정의
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8, 2, 10)))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (8, 3)                    42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


## 3. 파이썬으로 RNN 구현하기

In [11]:
# 초기 은닉 상태 출력
import numpy as np

timesteps = 10
input_dim = 4
hidden_units = 8    # 은닉 상태의 크기를 8로 지정하였으므로 8의 차원을 가지는 0의 값으로 구성된 벡터 출력

# 입력에 해당되는 2D 텐서
inputs = np.random.random((timesteps, input_dim))

# 초기 은닉 상태는 0(벡터)로 초기화
hidden_state_t = np.zeros((hidden_units,))

print('초기 은닉 상태 :', hidden_state_t)

초기 은닉 상태 : [0. 0. 0. 0. 0. 0. 0. 0.]


In [12]:
# 가중치와 편향을 각 크기에 맞게 정의하고 크기를 출력
Wx = np.random.random((hidden_units, input_dim))    # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치
Wh = np.random.random((hidden_units, hidden_units))    #(8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치
b = np.random.random((hidden_units,))    # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias)

print("가중치 Wx의 크기(shape) :", np.shape(Wx))
print("가중치 Wh의 크기(shape) :", np.shape(Wh))
print("편향의 크기(shape) :", np.shape(b))

가중치 Wx의 크기(shape) : (8, 4)
가중치 Wh의 크기(shape) : (8, 8)
편향의 크기(shape) : (8,)


In [13]:
total_hidden_states = []

# 각 시점 별 입력값
for input_t in inputs:

  # Wx * Xt + Wh * Ht-1 + b(bias)
  output_t = np.tanh(np.dot(Wx, input_t) + np.dot(Wh, hidden_state_t) + b)

  # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep t, output_dim)
  # 각 시점의 은닉 상태의 값을 계속해서 누적
  total_hidden_states.append(list(output_t))
  hidden_state_t = output_t

# 출력 시 값을 깔끔하게 해주는 용도
total_hidden_states = np.stack(total_hidden_states, axis = 0)

# (timesteps, output_dim)
print("모든 시점의 은닉 상태 :")
print(total_hidden_states)

모든 시점의 은닉 상태 :
[[0.85892501 0.73176634 0.5251392  0.89855182 0.80492195 0.55550789
  0.95886989 0.94872285]
 [0.99958636 0.99737525 0.99843232 0.99994022 0.9995729  0.99958485
  0.99978895 0.99989431]
 [0.99993836 0.99965306 0.99990156 0.99999652 0.99998131 0.99993636
  0.99998939 0.99998785]
 [0.99979967 0.99896374 0.99961893 0.99997422 0.99950914 0.99984453
  0.99995005 0.99997152]
 [0.99979573 0.99878326 0.9993622  0.99997098 0.99938293 0.99987047
  0.99994438 0.99997303]
 [0.9998951  0.99919469 0.99971106 0.99998333 0.99977733 0.99990134
  0.99998262 0.99998682]
 [0.99995261 0.99930951 0.99983762 0.99998804 0.99991495 0.99993583
  0.99999148 0.99999357]
 [0.99989408 0.99958168 0.99983894 0.99999529 0.99996173 0.99991781
  0.99997969 0.99997959]
 [0.99991443 0.99957662 0.9998786  0.9999931  0.99993164 0.99990308
  0.99999216 0.99998852]
 [0.99981444 0.99908011 0.9994009  0.99998702 0.99982257 0.99990953
  0.99991842 0.99996495]]


## 4. 깊은 순환 신경망

In [14]:
# 은닉층을 2개 추가하는 경우의 코드
model = Sequential()
model.add(SimpleRNN(hidden_units, input_length=10, input_dim=5, return_sequences=True))
model.add(SimpleRNN(hidden_units, return_sequences=True))

## 5. 양방향 순환 신경망

In [16]:
# 앞 시점의 은닉상태와 뒤 시점의 은닉 상태의 값 모두가 현재 시점의 출력층에서 출력값을 예측하기 위해 사용
from tensorflow.keras.models import Bidirectional

timesteps = 10
input_dim = 5

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape=(timesteps, input_dim)))

ImportError: ignored

# 4) 케라스의 SimpleRNN과 LSTM 이해하기
## 1. 임의의 입력 생성하기

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

In [20]:
# RNN과 LSTM을 테스트하기 위한 임의의 입력
train_X = [[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]
print(np.shape(train_X))

(4, 5)


In [21]:
# 2D 텐서를 3D 텐서로 변경
train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape)

(1, 4, 5)


## SimpleRNN 이해하기

In [22]:
# 은닉 상태의 크기3, 두 인자 값이 모두 False일 때의 출력값
rnn = SimpleRNN(3)
# rnn = SimpleRNN(3, return_sequences=False, return_state=False)와 동일
hidden_state = rnn(train_X)

print("hidden state : {}, shape : {}".format(hidden_state, hidden_state.shape))

hidden state : [[ 0.92126375 -0.9438475  -0.9928897 ]], shape : (1, 3)


In [23]:
# return_sequences를 True로 지정하여 모든 시점의 은닉 상태를 출력
rnn = SimpleRNN(3, return_sequences=True)
hidden_states = rnn(train_X)

print("hidden states : {}, shape : {}".format(hidden_states, hidden_states.shape))

hidden states : [[[ 0.9833128   0.9935881  -0.9993068 ]
  [ 0.85626465  0.99619716 -0.9940144 ]
  [-0.02891342  0.1666882  -0.925371  ]
  [-0.6240324   0.92511374 -0.93153566]]], shape : (1, 4, 3)


In [24]:
# return_state가 True일 경우에는 return_sequences의 True/False 여부와 상관없이 마지막 시점의 은닉 상태를 출력
rnn = SimpleRNN(3, return_sequences=True, return_state=True)
hidden_states, last_state = rnn(train_X)

print("hidden states : {}, shape : {}".format(hidden_states, hidden_states.shape))
print("last hidden state : {}, shape : {}".format(last_state, last_state.shape))

hidden states : [[[-0.21191558  0.9247593  -0.5584665 ]
  [-0.16361693  0.78625154 -0.96553063]
  [ 0.2464614   0.21260898 -0.97729856]
  [ 0.5551516   0.98431927 -0.49670163]]], shape : (1, 4, 3)
last hidden state : [[ 0.5551516   0.98431927 -0.49670163]], shape : (1, 3)


In [25]:
# return_sequences는 False, return_state는 True인 경우
# 두 개의 출력 모두 마지막 시점의 은닉 상태를 출력
rnn = SimpleRNN(3, return_sequences=False, return_state=True)
hidden_state, last_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))

hidden state : [[ 0.9712757  -0.9757286   0.04037877]], shape: (1, 3)
last hidden state : [[ 0.9712757  -0.9757286   0.04037877]], shape: (1, 3)


## LSTM 이해하기

In [26]:
# 임의의 입력에 대해서 LSTM을 사용할 경우
lstm = LSTM(3, return_sequences=False, return_state=True)
hidden_state, last_state, last_cell_state = lstm(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden state : [[-0.12563483 -0.30545184  0.09709059]], shape: (1, 3)
last hidden state : [[-0.12563483 -0.30545184  0.09709059]], shape: (1, 3)
last cell state : [[-0.65726584 -1.1308584   0.24708945]], shape: (1, 3)


In [27]:
# LSTM은 return_state를 True로 둔 경우에는 마지막 시점의 은닉 상태뿐만 아니라 셀 상태까지 반환
# return_sequences = True
lstm = LSTM(3, return_sequences=True, return_state=True)
hidden_states, last_hidden_state, last_cell_state = lstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state : {}, shape: {}'.format(last_hidden_state, last_hidden_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden states : [[[ 2.9361385e-01 -7.8314334e-02  6.8932626e-04]
  [ 3.1340003e-01 -4.3612391e-01 -3.3691153e-04]
  [ 4.2513573e-01 -5.2452689e-01  4.2103127e-02]
  [ 4.2195633e-01 -5.5606651e-01  4.5449071e-02]]], shape: (1, 4, 3)
last hidden state : [[ 0.42195633 -0.5560665   0.04544907]], shape: (1, 3)
last cell state : [[ 0.642616   -2.176991    0.06499276]], shape: (1, 3)


## 3. Bidirectional(LSTM) 이해하기

In [28]:
k_init = tf.keras.initializers.Constant(value=0.1)
b_init = tf.keras.initializers.Constant(value=0)
r_init = tf.keras.initializers.Constant(value=0.1)

In [29]:
# return_sequences가 False이고, return_state가 True인 경우
bilstm = Bidirectional(LSTM(3, return_sequences=False, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[0.63031393 0.63031393 0.63031393 0.7038734  0.7038734  0.7038734 ]], shape: (1, 6)
forward state : [[0.63031393 0.63031393 0.63031393]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


In [30]:
# 현재 은닉 상태의 값을 고정시켜두었기 때문에 return_sequences를 True로 할 경우, 출력이 어떻게 바뀌는지 비교 가능
bilstm = Bidirectional(LSTM(3, return_sequences=True, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[[0.35906473 0.35906473 0.35906473 0.7038734  0.7038734  0.7038734 ]
  [0.5511133  0.5511133  0.5511133  0.58863586 0.58863586 0.58863586]
  [0.59115744 0.59115744 0.59115744 0.3951699  0.3951699  0.3951699 ]
  [0.63031393 0.63031393 0.63031393 0.21942244 0.21942244 0.21942244]]], shape: (1, 4, 6)
forward state : [[0.63031393 0.63031393 0.63031393]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


# 6) RNN을 이용한 텍스트 생성
## 1. RNN을 이용하여 택스트 생성하기
### 1) 데이터에 대한 이해와 전처리

In [31]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [32]:
# 단어 집합을 생성하고 크기 확인
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print("단어 집합의 크기 : %d" % vocab_size)

단어 집합의 크기 : 12


In [33]:
# 각 단어와 단어에 부여된 정수 인덱스 출력
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [34]:
# 훈련 데이터 작성
sequences = list()
for line in text.split('\n'):    # 줄바꿈 문자를 기준으로 문장 토큰화
  encoded = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

print("학습에 사용할 샘플의 개수 : %d" %len(sequences))

학습에 사용할 샘플의 개수 : 11


In [35]:
# 전체 샘플 출력
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [36]:
# 가장 긴 샘플을 기준으로 전체 샘플의 길이 일치시키기
max_len = max(len(l) for l in sequences)    # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print("샘플의 최대 길이 : {}".format(max_len))

샘플의 최대 길이 : 6


In [37]:
# 가장 긴 샘플의 길이로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [38]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [39]:
# 각 샘플의 마지막 단어를 레이블로 분리
sequences = np.array(sequences)
x = sequences[:, :-1]     # 리스트의 마지막 값을 제외하고 저장
y = sequences[:, -1]    # 리스트의 마지막 값만 저장
print(x)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [40]:
# 레이블에 대해서 원-핫 인코딩 수행
y = to_categorical(y, num_classes = vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### 2) 모델 설계하기

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [43]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4750 - accuracy: 0.0909 - 1s/epoch - 1s/step
Epoch 2/200
1/1 - 0s - loss: 2.4625 - accuracy: 0.0909 - 9ms/epoch - 9ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4498 - accuracy: 0.0909 - 7ms/epoch - 7ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4369 - accuracy: 0.0909 - 5ms/epoch - 5ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4237 - accuracy: 0.0909 - 5ms/epoch - 5ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4101 - accuracy: 0.1818 - 6ms/epoch - 6ms/step
Epoch 7/200
1/1 - 0s - loss: 2.3959 - accuracy: 0.0909 - 8ms/epoch - 8ms/step
Epoch 8/200
1/1 - 0s - loss: 2.3812 - accuracy: 0.1818 - 8ms/epoch - 8ms/step
Epoch 9/200
1/1 - 0s - loss: 2.3659 - accuracy: 0.3636 - 6ms/epoch - 6ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3498 - accuracy: 0.3636 - 5ms/epoch - 5ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3329 - accuracy: 0.4545 - 5ms/epoch - 5ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3152 - accuracy: 0.4545 - 7ms/epoch - 7ms/step
Epoch 13/200
1/1 - 0s - loss: 2.2967 - accuracy: 0.4545 - 6ms/e

In [49]:
# 모델이 정확하게 예측하고 있는지 문장을 생성하는 함수
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [50]:
print(sentence_generation(model, tokenizer, "경마장에", 4))

경마장에 있는 말이 뛰고 있다


In [51]:
print(sentence_generation(model, tokenizer, '그의', 2))

그의 말이 법이다


In [52]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는 말이 고와야 오는 말이 곱다


## 2. LSTM을 이용하여 텍스트 생성하기
### 1) 데이터에 대한 이해와 전처리

In [53]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [54]:
df = pd.read_csv("/content/ArticlesApril2018.csv")
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [55]:
print("열의 개수 :", len(df.columns))
print(df.columns)

열의 개수 : 15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [56]:
print(df['headline'].isnull().values.any())    # NULL값 있는지 확인

False


In [57]:
headline = []
# 헤드라인의 값들을 리스트로 저장
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [58]:
print("총 샘플의 개수 : {}".format(len(headline)))

총 샘플의 개수 : 1324


In [61]:
# Unkown값을 가진 샘플 제거
headline = [word for word in headline if word != 'Unknown']
print("노이즈값 제거 후 샘플의 개수 : {}".format(len(headline)))

노이즈값 제거 후 샘플의 개수 : 1214


In [62]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [64]:
# 데이터 전처리 수행
def repreprocessing(raw_sentence):
  preproceseed_sentence = raw_sentence.encode('utf8').decode('ascii', 'ignore')
  # 구두점 제거와 동시에 소문자화
  return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [65]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print("단어 집합의 크기 : %d" %vocab_size)

단어 집합의 크기 : 3494


In [67]:
sequences = list()

for sentence in preprocessed_headline:

    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0] 
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [68]:
# 인덱스로부터 단어를 찾는 index_to_word
index_to_word = {}
for key, value in tokenizer.word_index.items():    # 인덱스를 단어로 바꾸기 위해 index_to_word 생성
  index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [69]:
# 가장 긴 샘플의 길이 확인
max_len = max(len(l) for l in sequences)
print("샘플의 최대 길이 : {}".format(max_len))

샘플의 최대 길이 : 24


In [70]:
# 가장 긴 샘플의 길이로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [73]:
# 맨 우측 단어만 레이블로 분리
sequences = np.array(sequences)
x = sequences[:, :-1]
y = sequences[:, -1]

# 맨 우측의 정수값이 사라져 샘플의 길이가 24에서 23이 됨
print(x[:3])
print(y[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]
[ 269  371 1115]


In [74]:
# 레이블 데이터 y에 대하여 원-핫 인코딩
y = to_categorical(y, num_classes=vocab_size)

### 2) 모델 설계하기

In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [76]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 10s - loss: 7.6339 - accuracy: 0.0283 - 10s/epoch - 41ms/step
Epoch 2/200
244/244 - 8s - loss: 7.1070 - accuracy: 0.0327 - 8s/epoch - 31ms/step
Epoch 3/200
244/244 - 8s - loss: 6.9705 - accuracy: 0.0322 - 8s/epoch - 31ms/step
Epoch 4/200
244/244 - 8s - loss: 6.9864 - accuracy: 0.0379 - 8s/epoch - 31ms/step
Epoch 5/200
244/244 - 8s - loss: 6.7248 - accuracy: 0.0437 - 8s/epoch - 32ms/step
Epoch 6/200
244/244 - 8s - loss: 6.5663 - accuracy: 0.0451 - 8s/epoch - 31ms/step
Epoch 7/200
244/244 - 7s - loss: 6.3933 - accuracy: 0.0514 - 7s/epoch - 31ms/step
Epoch 8/200
244/244 - 7s - loss: 6.2026 - accuracy: 0.0569 - 7s/epoch - 31ms/step
Epoch 9/200
244/244 - 7s - loss: 6.0112 - accuracy: 0.0625 - 7s/epoch - 31ms/step
Epoch 10/200
244/244 - 8s - loss: 5.8309 - accuracy: 0.0655 - 8s/epoch - 31ms/step
Epoch 11/200
244/244 - 8s - loss: 5.6584 - accuracy: 0.0684 - 8s/epoch - 31ms/step
Epoch 12/200
244/244 - 8s - loss: 5.4968 - accuracy: 0.0716 - 8s/epoch - 31ms/step
Epoch 13/20

In [77]:
# 문장을 생성하는 함수
def sentence_generation(model, tokenizer, current_word, n):  # 모델, 토크나이저, 현재 단어, 반복할 횟수
  init_word = current_word
  sentence = ''

  # n번 반복
  for _ in range(n):
    encoded = tokenizer.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

    # 입력한 x(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장
    result = model.predict(encoded, verbose=0)
    result = np.argmax(result, axis=1)

    for word, index in tokenizer.word_index.items():
      # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
      if index == result:
        break

    # 현재 단어 + ' ' + 에측 단어를 현재 단어로 변경
    current_word = current_word + ' ' + word

    # 예측 단어를 문장에 저장
    sentence = sentence + ' ' + word

  sentence = init_word + sentence
  return sentence

In [78]:
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성
print(sentence_generation(model, tokenizer, 'i', 10))

# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성
print(sentence_generation(model, tokenizer, 'how', 10))

i cant jump ship from facebook yet own pruitt a trail
how to make facebook more accountable are on far the tip


# 7) 문자 단위 RNN
## 1. 문자 단위 RNN 언어 모델
### 1) 데이터에 대한 이해와 전처리

In [79]:
# 데이터 로드, 특수문자 제거, 단어 소문자화
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

# 데이터 로드
urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="11-0.txt")

f = open('11-0.txt', 'rb')
sentences = []
for sentence in f:    # 데이터로부터 한 줄씩 읽는다
  sentence = sentence.strip()    # strip()을 통해 \r, \n을 제거한다
  sentence = sentence.lower()    # 소문자화
  sentence = sentence.decode('ascii', 'ignore')    # \xe2\x80\x99 등과 같은 바이트 열 제거
  if len(sentence) > 0:
    sentences.append(sentence)

f.close()

In [80]:
sentences[:5]

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. you may copy it, give it away or re-use it under the terms',
 'of the project gutenberg license included with this ebook or online at']

In [81]:
# 하나의 문자열로 통합
total_data = ' '.join(sentences)
print("문자열의 길이 또는 총 문자의 개수 : %d" % len(total_data))

문자열의 길이 또는 총 문자의 개수 : 159484


In [82]:
print(total_data[:200])

the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with


In [83]:
# 위의 문자열로부터 문자 집합
char_vocab = sorted(list(set(total_data)))
vocab_size = len(char_vocab)
print("문자 집합의 크기 : {}".format(vocab_size))

문자 집합의 크기 : 56


In [84]:
# 문자에 고유한 정수 부여
char_to_index = dict((char, index) for index, char in enumerate(char_vocab))
print("문자 집합 :", char_to_index)

문자 집합 : {' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '[': 27, ']': 28, '_': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55}


In [85]:
# 정수로부터 문자를 리턴하는 index_to_char 작성
index_to_char = {}
for key, value in char_to_index.items():
  index_to_char[value] = key

In [86]:
# appl(입력 시퀀스) -> pple(예측해야 하는 시퀀스)
train_x = 'appl'
train_y = 'pple'

In [87]:
seq_length = 60

# 문자열의 길이를 seq_length로 나누면 전처리 후 생겨날 샘플 수
n_samples = int(np.floor((len(total_data) - 1) / seq_length))
print("샘플의 수 : {}".format(n_samples))

샘플의 수 : 2658


In [88]:
train_x = []
train_y = []

for i in range(n_samples):
  # 0:60 -> 60:120 -> 120:180로 loop를 돌면서 문장 샘플을 1개씩 pick
  x_sample = total_data[i * seq_length: (i+1) * seq_length]

  # 정수 인코딩
  x_encoded = [char_to_index[c] for c in x_sample]
  train_x.append(x_encoded)

  # 오른쪽으로 1칸 쉬프트
  y_sample = total_data[i * seq_length + 1: (i+1) * seq_length + 1]
  y_encoded = [char_to_index[c] for c in y_sample]
  train_y.append(y_encoded)

In [90]:
print('x 데이터의 첫번째 샘플 :', train_x[0])
print('y 데이터의 첫번째 샘플 :', train_y[0])
print('-'*50)
print('x 데이터의 첫번째 샘플 디코딩 :', [index_to_char[i] for i in train_x[0]])
print('y 데이터의 첫번째 샘플 디코딩 :', [index_to_char[i] for i in train_y[0]])

x 데이터의 첫번째 샘플 : [49, 37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30]
y 데이터의 첫번째 샘플 : [37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30, 43]
--------------------------------------------------
x 데이터의 첫번째 샘플 디코딩 : ['t', 'h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e', 'b', 'o', 'o', 'k', ' ', 'o', 'f', ' ', 'a', 'l', 'i', 'c', 'e', 's', ' ', 'a', 'd', 'v', 'e', 'n', 't', 'u', 'r', 'e', 's', ' ', 'i', 'n', ' ', 'w', 'o', 'n', 'd', 'e', 'r', 'l', 'a']
y 데이터의 첫번째 샘플 디코딩 : ['h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e',

In [91]:
#train_y[1]은 train_x[1]에서 오른쪽으로 한 칸 쉬프트 된 문장
print(train_x[1])

[43, 33, 10, 0, 31, 54, 0, 41, 34, 52, 38, 48, 0, 32, 30, 47, 47, 44, 41, 41, 0, 49, 37, 38, 48, 0, 34, 31, 44, 44, 40, 0, 38, 48, 0, 35, 44, 47, 0, 49, 37, 34, 0, 50, 48, 34, 0, 44, 35, 0, 30, 43, 54, 44, 43, 34, 0, 30, 43, 54]


In [92]:
print(train_y[1])

[33, 10, 0, 31, 54, 0, 41, 34, 52, 38, 48, 0, 32, 30, 47, 47, 44, 41, 41, 0, 49, 37, 38, 48, 0, 34, 31, 44, 44, 40, 0, 38, 48, 0, 35, 44, 47, 0, 49, 37, 34, 0, 50, 48, 34, 0, 44, 35, 0, 30, 43, 54, 44, 43, 34, 0, 30, 43, 54, 52]


In [93]:
train_x = to_categorical(train_x)
train_y = to_categorical(train_y)

print('train_x의 크기(shape) : {}'.format(train_x.shape))    # 원-핫 인코딩
print('train_y의 크기(shape) : {}'.format(train_y.shape))    # 원-핫 인코딩

train_x의 크기(shape) : (2658, 60, 56)
train_y의 크기(shape) : (2658, 60, 56)


### 2) 모델 설계하기

In [94]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

hidden_units = 256

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(None, train_x.shape[2]), return_sequences=True))
model.add(LSTM(hidden_units, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=80, verbose=2)

Epoch 1/80
84/84 - 45s - loss: 3.0681 - accuracy: 0.1834 - 45s/epoch - 541ms/step
Epoch 2/80
84/84 - 42s - loss: 2.8110 - accuracy: 0.2270 - 42s/epoch - 500ms/step
Epoch 3/80
84/84 - 42s - loss: 2.4642 - accuracy: 0.3113 - 42s/epoch - 501ms/step
Epoch 4/80
84/84 - 42s - loss: 2.3390 - accuracy: 0.3375 - 42s/epoch - 498ms/step
Epoch 5/80
84/84 - 42s - loss: 2.2386 - accuracy: 0.3622 - 42s/epoch - 497ms/step
Epoch 6/80
84/84 - 42s - loss: 2.1563 - accuracy: 0.3844 - 42s/epoch - 500ms/step
Epoch 7/80
84/84 - 43s - loss: 2.1035 - accuracy: 0.3972 - 43s/epoch - 507ms/step
Epoch 8/80
84/84 - 43s - loss: 2.0453 - accuracy: 0.4118 - 43s/epoch - 506ms/step
Epoch 9/80
84/84 - 43s - loss: 1.9949 - accuracy: 0.4233 - 43s/epoch - 509ms/step
Epoch 10/80
84/84 - 43s - loss: 1.9552 - accuracy: 0.4355 - 43s/epoch - 508ms/step
Epoch 11/80
84/84 - 43s - loss: 1.9194 - accuracy: 0.4434 - 43s/epoch - 507ms/step
Epoch 12/80
84/84 - 42s - loss: 1.8807 - accuracy: 0.4539 - 42s/epoch - 501ms/step
Epoch 13/80
8

In [95]:
# 특정 문자를 주면 다음 문자를 계속해서 생성
def sentence_generation(model, length):
  # 문자에 대한 랜덤한 정수 생성
  ix = [np.random.randint(vocab_size)]

  # 랜덤한 정수로부터 맵핑되는 문자 생성
  y_char = [index_to_char[ix[-1]]]
  print(ix[-1], '번 문자', y_char[-1], '로 예측을 시작!')

  # (1, length, 55) 크기의 x 생성. 즉, LSTM의 입력 시퀀스 생성
  x = np.zeros((1, length, vocab_size))

  for i in range(length):
    # x[0][i][예측한 문자의 인덱스] = 1, 즉, 예측 문자를 다음 입력 시퀀스에 추가
    x[0][i][ix[-1]] = 1
    print(index_to_char[ix[-1]], end="")
    ix = np.argmax(model.predict(x[:, :i+1, :])[0], 1)
    y_char.append(index_to_char[ix[-1]])
  return ('').join(y_char)

In [96]:
result = sentence_generation(model, 100)
print(result)

18 번 문자 4 로 예측을 시작!
4 down the rabbit-hole chapter ii.    the pool of tears chapter viii m glowed oo manne my waist dawa4 down the rabbit-hole chapter ii.    the pool of tears chapter viii m glowed oo manne my waist dawal


## 2. 문자 단위 RNN으로 텍스트 생성하기
### 1) 데이터에 대한 이해와 전처리

In [97]:
import numpy as np
from tensorflow.keras.utils import to_categorical

In [98]:
raw_text = '''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

# 위에는 저자가 작성한 노래 가사
# 위에 텍스트에 존재하는 단락 구분을 없애고 하나의 문자열로 재저장
tokens = raw_text.split()
raw_text = ' '.join(tokens)
print(raw_text)

I get on with life as a programmer, I like to contemplate beer. But when I start to daydream, My mind turns straight to wine. Do I love wine more than beer? I like to use words about beer. But when I stop my talking, My mind turns straight to wine. I hate bugs and errors. But I just think back to wine, And I'm happy once again. I like to hang out with programming and deep learning. But when left alone, My mind turns straight to wine.


In [99]:
# 중복을 제거한 문자 집합 생성
char_vocab = sorted(list(set(raw_text)))
vocab_size = len(char_vocab)
print("문자 집합 :", char_vocab)
print("문자 집합의 크기 : {}".format(vocab_size))

문자 집합 : [' ', "'", ',', '.', '?', 'A', 'B', 'D', 'I', 'M', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']
문자 집합의 크기 : 33


In [100]:
char_to_index = dict((char, index) for index, char in enumerate(char_vocab))    #문자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}


In [101]:
# 입력 시퀀스의 길이가 10이 되도록 데이터 구성
# 예측 대상인 문자도 필요하므로 길이가 11이 되도록 데이터 구성
length = 11
sequences = []
for i in range(length, len(raw_text)):
  seq = raw_text[i-length:i]    # 길이 11의 문자열을 지속적으로 만든다
  sequences.append(seq)
print('총 훈련 샘플의 수 : %d' % len(sequences))

총 훈련 샘플의 수 : 426


In [102]:
# 10개만 출력
sequences[:10]

['I get on wi',
 ' get on wit',
 'get on with',
 'et on with ',
 't on with l',
 ' on with li',
 'on with lif',
 'n with life',
 ' with life ',
 'with life a']

In [103]:
# char_to_index(앞에서 만듬)를 사용하여 전체 데이터에 대해서 정수 인코딩을 수행
encoded_sequences = []
for sequence in sequences:    # 전체 데이터에서 문장 샘플을 1개씩 꺼낸다
  encoded_sequence = [char_to_index[char] for char in sequence]    # 문장 샘플에서 각 문자에 대해서 정수 인코딩을 수행
  encoded_sequences.append(encoded_sequence)

In [104]:
# 정수 인코딩 된 결과가 x에 저장됨. 5개만 출력
encoded_sequences[:5]

[[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18],
 [0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28],
 [16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17],
 [14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0],
 [28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]]

In [105]:
# 문자 분리
# 모든 샘플 문장에 대해서 마지막 문자를 분리하여
# 마지막 문자가 분리된 샘플은 x_data에 저장, 마지막 문자는 y_data에 저장
encoded_sequences = np.array(encoded_sequences)

# 맨 마지막 위치의 문자를 분리
x_data = encoded_sequences[:, :-1]
# 맨 마지막 위치의 문자를 저장
y_data = encoded_sequences[:, -1]

print(x_data[:5])
print(y_data[:5])

[[ 8  0 16 14 28  0 24 23  0 31]
 [ 0 16 14 28  0 24 23  0 31 18]
 [16 14 28  0 24 23  0 31 18 28]
 [14 28  0 24 23  0 31 18 28 17]
 [28  0 24 23  0 31 18 28 17  0]]
[18 28 17  0 21]


In [106]:
# x와 y에 대해서 원-핫 인코딩
x_data_one_hot = [to_categorical(encoded, num_classes=vocab_size) for encoded in x_data]
x_data_one_hot = np.array(x_data_one_hot)
y_data_one_hot = to_categorical(y_data, num_classes=vocab_size)

print(x_data_one_hot.shape)

(426, 10, 33)


### 2) 모델 설계하기

In [107]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

hidden_units = 64

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(x_data_one_hot.shape[1], x_data_one_hot.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_data_one_hot, y_data_one_hot, epochs=100, verbose=2)

Epoch 1/100
14/14 - 2s - loss: 3.4600 - accuracy: 0.1479 - 2s/epoch - 137ms/step
Epoch 2/100
14/14 - 0s - loss: 3.3193 - accuracy: 0.1972 - 95ms/epoch - 7ms/step
Epoch 3/100
14/14 - 0s - loss: 3.0778 - accuracy: 0.1972 - 97ms/epoch - 7ms/step
Epoch 4/100
14/14 - 0s - loss: 3.0087 - accuracy: 0.1972 - 97ms/epoch - 7ms/step
Epoch 5/100
14/14 - 0s - loss: 2.9702 - accuracy: 0.1972 - 96ms/epoch - 7ms/step
Epoch 6/100
14/14 - 0s - loss: 2.9456 - accuracy: 0.1972 - 94ms/epoch - 7ms/step
Epoch 7/100
14/14 - 0s - loss: 2.9266 - accuracy: 0.1972 - 96ms/epoch - 7ms/step
Epoch 8/100
14/14 - 0s - loss: 2.9161 - accuracy: 0.1972 - 98ms/epoch - 7ms/step
Epoch 9/100
14/14 - 0s - loss: 2.8987 - accuracy: 0.1972 - 109ms/epoch - 8ms/step
Epoch 10/100
14/14 - 0s - loss: 2.8828 - accuracy: 0.1972 - 98ms/epoch - 7ms/step
Epoch 11/100
14/14 - 0s - loss: 2.8646 - accuracy: 0.1972 - 101ms/epoch - 7ms/step
Epoch 12/100
14/14 - 0s - loss: 2.8502 - accuracy: 0.1972 - 96ms/epoch - 7ms/step
Epoch 13/100
14/14 - 0s

In [108]:
# 문장을 생성하는 함수
def sentence_generation(model, char_to_index, seq_length, seed_text, n):
  # 초기 시퀀스
  init_text = seed_text
  sentence = ''

  #다음 문자 예측은 총 n번만 반복
  for _ in range(n):
    encoded = [char_to_index[char] for char in seed_text]  # 현재 시퀀스에 대한 정수 인코딩
    encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre')  # 데이터에 대한 패딩
    encoded = to_categorical(encoded, num_classes=len(char_to_index))

    # 입력한 x(현재 시퀀스)에 대해서 y를 예측하고 y(예측한 문자)를 result에 저장
    result = model.predict(encoded, verbose=0)
    result = np.argmax(result, axis=1)

    for char, index in char_to_index.items():
      if index == result:
        break

    # 현재 시퀀스 + 예측 문자를 현재 시퀀스로 변경
    seed_text = seed_text + char

    # 예측 문자를 문장에 저장
    sentence = sentence + char

  # n번의 다음 문자 예측이 끝나면 최종 완성된 문장을 리턴
  sentence = init_text + sentence
  return sentence

In [109]:
print(sentence_generation(model, char_to_index, 10, 'I get on w', 80))

I get on with life as a programmer, I like to hang out with prorrammiug I ndndt tayll...  
